In [1]:
from ris import pysqldb
from IPython.display import clear_output
from collections import defaultdict
from collections import defaultdict, namedtuple
import datetime 
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

print 'Notebook run: {}'.format(timestamp)
print os.getcwd()

Notebook run: 2020-02-10_17:48
E:\RIS\Staff Folders\Samuel\Requests\SH\Refrence_Marker_Crash_Request


In [2]:
sqldb = pysqldb.DbConnect(server='dot55sql01', database='forms', type='MS', user = 'arcgis', password = 'arcgis')

Database connection (MS) to forms on dot55sql01 - user: arcgis 
Connection established 2020-02-10 17:48:03
Query run in 2000 microseconds


In [3]:
sqldb.connect()

Database connection (MS) to forms on dot55sql01 - user: arcgis 
Connection established 2020-02-10 17:48:04


### Retrieving all Crash and Vehicular Data within Reference Marker Study Area

In [4]:
data = sqldb.dfquery("""    


WITH data AS(
--Retrieving all vehicle level data (Vehicle Type, State of registration, Number of Occupants, Drivers Age, Sex, 
--Direction of Travel, Public Property Damage, Pre-Accident Action and Apparent Factors. 
Select core.OFF_STREET
      ,core.integration_id
      ,core.ACCIDENT_DT 
      ,row_number() OVER (PARTITION BY core.INTEGRATION_ID ORDER BY core.ACCIDENT_DT) AS VEH -- Assigning a row number for vehicle per crash
      ,veh.[ST_OF_REG]
      ,veh.VEHICLE_TYPE_CODE
      ,veh.[VEH_OCCUPANTS]
      ,veh.VICTIM_AGE DRIVER_AGE
      ,veh.[DRIVER_SEX]
      ,veh.[VIOLATION]
      ,veh.[DIRECTION_OF_TRAVEL]
      ,veh.[PROPERTY_DAMAGED_DESC]
      ,veh.[CONTRIBUTING_FACTOR]
      ,veh.[CONTRIBUTING_FACTOR_2]
      ,veh.[PRE_ACDNT_ACTION]
FROM [FORMS].[dbo].[WC_ACCIDENT_F] core
JOIN (SELECT DISTINCT veh.*,vic.VICTIM_AGE 
      FROM FORMS.dbo.WC_ACCIDENT_VEHICLE_F veh 
      --Joining to Victim Table to retrieve victim ages
      LEFT JOIN (SELECT ACCIDENT_ID, VEHICLE_NUM, VICTIM_AGE   
                 FROM WC_ACCIDENT_VICTIM_F vic
                 WHERE PERSON_ROLE_CODE = 'Driver' -- Only interested in injured driver's age
                ) vic
      ON veh.ACCIDENT_ID = vic.ACCIDENT_ID and veh.VEHICLE_NUM = vic.VEHICLE_NUM) veh
ON core.integration_id = veh.accident_id
WHERE core.OFF_STREET in( '907LX4M12094') --Reference marker study area
and YEAR(core.ACCIDENT_DT) between 2017 and 2019 
)

SELECT core_data.*
      ,veh_data.INTEGRATION_ID
      ,veh_data.VEH
      ,veh_data.[ST_OF_REG]
      ,veh_data.[VEHICLE_TYPE_CODE]
      ,veh_data.[VEH_OCCUPANTS]
      ,veh_data.DRIVER_AGE
      ,veh_data.[DRIVER_SEX]
      ,veh_data.[VIOLATION]
      ,veh_data.[DIRECTION_OF_TRAVEL]
      ,veh_data.[PROPERTY_DAMAGED_DESC]
      ,veh_data.[CONTRIBUTING_FACTOR]
      ,veh_data.[CONTRIBUTING_FACTOR_2]
      ,veh_data.[PRE_ACDNT_ACTION]
FROM (--Retrieving all crash level data (Reference Marker, Street, Accident Date, Time, Injured count, Accident Class, Number of Vehicles, Manner of Collision, Traffic Control   
      --Road Surface Condition, Road Character, Light Condition. 
      SELECT core.OFF_STREET
            ,core.integration_id
            ,row_number() OVER (PARTITION BY core.OFF_STREET ORDER BY core.integration_id) AS case_num -- Giving a row number to each distinct integration id 
            ,core.ACCIDENT_DT
            ,left(stuff(STUFF(right('000000'+cast(core.ACCIDENT_TIME_WID AS VARCHAR),6),5,0,':'),3,0,':'), 5) ACCIDENT_TIME -- Converting Accident Time field to proper time format
            ,core.SRC_CROSS_STREET
            ,core.SRC_ON_STREET
            ,core.SRC_DISTANCE_FROM_CROSS_STREET 
            ,core.SRC_NEAREST_LANDMARK
            ,core.SRC_DIRECTION_FROM_X_STREET
            ,CASE WHEN INJURED_CNT = 0 THEN 'Propetry Damage' ELSE 'Injury' END ACCIDENT_CLASS
            ,core.INJURED_CNT
            ,core.VEHICLE_INVOLVED_CNT
            ,core.FIRST_EVENT_TYPE 
            ,core.TRAFFIC_CONTROL
            ,core.ACCIDENT_DIAGRAM
            ,core.ROADWAY_SURFACE_COND
            ,core.ROADWAY_CHARACTER
            ,core.LIGHT_CONDITIONS
            ,core.WEATHER
            ,core.ACCIDENT_DESC
      FROM [FORMS].[dbo].[WC_ACCIDENT_F] core 
      WHERE core.OFF_STREET in( '907LX4M12094') --Reference marker study area
      and YEAR(core.ACCIDENT_DT) between 2017 and 2019
      ) core_data
JOIN (SELECT * FROM data) veh_data
ON core_data.INTEGRATION_ID = veh_data.integration_id
ORDER BY core_data.OFF_STREET, case_num


    """)

In [5]:
data = data.drop(columns=['case_num'])

### Excel Data Formatting

In [6]:
writer = pd.ExcelWriter(r'C:\Users\SHostetter\Desktop\reference_marker_crashes.xlsx',engine = 'xlsxwriter') #opening new excel file to write data to

In [7]:
i = 0 #iterator for unique integration ids
j = 0 #iterator for retrieving vehicle information for unique integration ids. 
count=0 #iterator to track location of empty rows to write to in excel.

while i < range(len(data.integration_id.unique())-1):
    while j<len(data):
        try:
            if data.integration_id[j] == data.integration_id.unique()[i]: #case to locate unique integration ids
                
                data.iloc[j:j+1,0:20].to_excel(writer, sheet_name='Sheet1', startrow=count, index = False) #writing crash level information to excel 
                
                count+=2 #location of next empty row in excel after writing crash level information
               
                veh_cnt = veh_cnt = int(data.loc[data.integration_id == data.iloc[j].integration_id].VEHICLE_INVOLVED_CNT.head(1)) #obtaining count of vehicles invloved in crash 
                data.iloc[j:j+veh_cnt,21::].to_excel(writer, sheet_name='Sheet1', startcol=2, startrow=count,index = False) #writing vehicle level information to excel       
                
                i+=1 #stepping to next unique integration id
                count+=veh_cnt+1 #location 
            
            j+=1 #stepping to next row in dataset
            
        except:
            break
        
    break
    
writer.save() #saving excel file 
